# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#langchain library
!pip install langchain==0.1.0
#LLM
!pip install accelerate==0.25.0
!pip install transformers==4.36.2
!pip install bitsandbytes==0.41.2
#Text Embedding
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding==1.0.1
#vectorstore
!pip install pymupdf==1.23.8
!pip install faiss-gpu==1.7.2
!pip install faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [5]:
from langchain import PromptTemplate

prompt_template = """
    I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.
    Whether it's about probabilistic models, language models, or any other related topic,
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!

    Context: {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).\n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.\n    Whether it's about probabilistic models, language models, or any other related topic,\n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n\n    Context: {context}\n    Question: {question}\n    Answer:")

In [6]:
PROMPT.format(
    context = "About AIT",
    question = "What is Machine Learning"
)

"I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).\n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.\n    Whether it's about probabilistic models, language models, or any other related topic,\n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n\n    Context: About AIT\n    Question: What is Machine Learning\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [7]:
file_path = '/content/drive/MyDrive/Colab Notebooks/NLP/A7'

In [8]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = f'{file_path}/data/pdf/Student-Handbook_August-2021.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [9]:
# documents

In [10]:
len(documents)

95

In [11]:
documents[1]

Document(page_content=' \n \n \nPREFACE \n \nThe Student Handbook will help you cope with your academic and social life at AIT. \nThere is no doubt that you will get much more if you are well informed on AIT \nregulations, services and facilities. \n \nThis handbook is up-to-date at the time of publishing. However, changes in regulations \nand procedures may be made before the next edition of the Student Handbook is \npublished. Important changes will be announced via email. \nBe well informed and make the best of your life at AIT.  \n \nOffice of Student Affairs  \nAugust 2022\n', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'page': 1, 'total_pages': 95, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'RICOH MP C3004ex', 'producer': 'RICOH MP C3004ex', 'creationDate': "D:20220

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [13]:
doc[1]

Document(page_content='TABLE OF CONTENTS \n \n \nI. \nIntroducing AIT \n \nII. \nStudent Bill of Rights \n \nIII. \nStudent Code of Conduct \n \nIV. \nGuidance for New Students \n \nV. \nStudent Welfare Unit and the AIT Career Center \n \nVI. \nHarassment Policy \n \nVII. \nSubstance Abuse Policy \n \nVIII. \nEnvironment Policy \n \nIX. \nStudent Organizations and Student Participation in Institute Governance \n- Student Union \n- Nationality Associations \n- Student Participation in Institute Governance \n \nX. \nCampus Facilities and Services \n- Student Accommodation \n- Visas \n- Banking \n- Dining \n- Sports and Recreation \n- Movies on Campus \n- Religious Services \n- Mails \n- AIT Reception \n- Office of Public Affairs (OPA)', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'page': 2, 'total_pages': 95, 'format': 

In [14]:
len(doc)

367

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [ ]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [16]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [17]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [18]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [19]:
#ready to use
retriever = vectordb.as_retriever()

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [20]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [21]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [22]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [24]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [25]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [26]:
file_path

'/content/drive/MyDrive/Colab Notebooks/NLP/A7'

In [30]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/A7/model
!git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

/content/drive/MyDrive/Colab Notebooks/NLP/A7/model
Cloning into 'fastchat-t5-3b-v1.0'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), 8.84 KiB | 14.00 KiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/Colab Notebooks/NLP/A7/model/fastchat-t5-3b-v1.0/.git/hooks/post-checkout': Permission denied
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model.bin

See: `git lfs help smudge` for more details.


In [31]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = f'{file_path}/model/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [32]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [33]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [34]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [35]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [36]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).\n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.\n    Whether it's about probabilistic models, language models, or any other related topic,\n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n\n    Context: {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7ab66c1d4490>)), document_variable_name='context')

In [37]:
query = "What is AIT?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.
    Whether it's about probabilistic models, language models, or any other related topic,
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!

    Context: a self-contained international community with a cosmopolitan approach to living and 
learning. 
 
Since 1959, AIT has carried out its mission “to develop highly qualified and committed 
professionals who play a leading role in the region’s sustainable development and its 
integration into the global economy” by supporting technological change 

{'input_documents': [Document(page_content='a self-contained international community with a cosmopolitan approach to living and \nlearning. \n \nSince 1959, AIT has carried out its mission “to develop highly qualified and committed \nprofessionals who play a leading role in the region’s sustainable development and its \nintegration into the global economy” by supporting technological change and \nsustainable development through higher learning, research, capacity building and \noutreach. \n \nAIT’s renowned degree programs are administered by its School of Engineering and \nTechnology; School of Environment, Resources and Development; and School of \nManagement. Students benefit from challenging academic programs and exciting', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'page': 3, 'total_pages': 95, 'format': 'PDF 1.

In [38]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).\n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.\n    Whether it's about probabilistic models, language models, or any other related topic,\n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n\n    Context: {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation

## 5. Chatbot

In [39]:
prompt_question = "Who is AIT's president?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.
    Whether it's about probabilistic models, language models, or any other related topic,
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!

    Context: requirements made under the authority of this policy. 
5. In discharging his/her responsibility to implement this Policy, the President may 
be assisted by the Campus Environment and Development Committee. 
6. The Institute Master Plan is now four decades old, and no longer a relevant 
gui

{'question': "Who is AIT's president?",
 'chat_history': [],
 'answer': "<pad> The  text  does  not  provide  any  information  about  AIT's  president.  The  President  is  the  head  of  the  AIT  Student  Union,  which  is  responsible  for  implementing  policies  and  ensuring  the  well-being  of  the  students.  The  Vice  President  becomes  President  of  the  SU  in  the  succeeding  semester.\n",
 'source_documents': [Document(page_content='requirements made under the authority of this policy. \n5. In discharging his/her responsibility to implement this Policy, the President may \nbe assisted by the Campus Environment and Development Committee. \n6. The Institute Master Plan is now four decades old, and no longer a relevant \nguide for the development of the campus. A task force has been appointed by \nthe President to update the plan, providing a framework for enabling AIT to \ndevelop its campus as a model of good environmental practice and governance, \nand a framework fo

In [40]:
prompt_question = "Where is AIT? What is its address"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content="Who is AIT's president?"), AIMessage(content="<pad> The  text  does  not  provide  any  information  about  AIT's  president.  The  President  is  the  head  of  the  AIT  Student  Union,  which  is  responsible  for  implementing  policies  and  ensuring  the  well-being  of  the  students.  The  Vice  President  becomes  President  of  the  SU  in  the  succeeding  semester.\n")]
Follow Up Input: Where is AIT? What is its address
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural L

{'question': 'Where is AIT? What is its address',
 'chat_history': [HumanMessage(content="Who is AIT's president?"),
  AIMessage(content="<pad> The  text  does  not  provide  any  information  about  AIT's  president.  The  President  is  the  head  of  the  AIT  Student  Union,  which  is  responsible  for  implementing  policies  and  ensuring  the  well-being  of  the  students.  The  Vice  President  becomes  President  of  the  SU  in  the  succeeding  semester.\n")],
 'answer': '<pad>  P.O.  Box  4  Klong  Luang  Pathumthani  12120,  Thailand\n',
 'source_documents': [Document(page_content='·         Main Post Office is located just outside the campus operates from \n8:30 a.m. to 4:00 p.m. (tel.: 02-516-8383). \n[O1]  Departing Persons \nAll persons leaving AIT permanently are advised to leave their forwarding \naddress with the AITCMS prior to their departure, if they are expecting any mail \nso that mail can be promptly forwarded to them instead of "return to sender." \nFormat 

In [41]:
prompt_question = "How many students and faculties are in AIT?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content="Who is AIT's president?"), AIMessage(content="<pad> The  text  does  not  provide  any  information  about  AIT's  president.  The  President  is  the  head  of  the  AIT  Student  Union,  which  is  responsible  for  implementing  policies  and  ensuring  the  well-being  of  the  students.  The  Vice  President  becomes  President  of  the  SU  in  the  succeeding  semester.\n"), HumanMessage(content='Where is AIT? What is its address'), AIMessage(content='<pad>  P.O.  Box  4  Klong  Luang  Pathumthani  12120,  Thailand\n')]
Follow Up Input: How many students and faculties are in AIT?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMC

{'question': 'How many students and faculties are in AIT?',
 'chat_history': [HumanMessage(content="Who is AIT's president?"),
  AIMessage(content="<pad> The  text  does  not  provide  any  information  about  AIT's  president.  The  President  is  the  head  of  the  AIT  Student  Union,  which  is  responsible  for  implementing  policies  and  ensuring  the  well-being  of  the  students.  The  Vice  President  becomes  President  of  the  SU  in  the  succeeding  semester.\n"),
  HumanMessage(content='Where is AIT? What is its address'),
  AIMessage(content='<pad>  P.O.  Box  4  Klong  Luang  Pathumthani  12120,  Thailand\n')],
 'answer': '<pad>   pad>  1,200  +  from  40+  countries\n                                                                                                                                                                                                                                               ',
 'source_documents': [Document(page_content='a self-containe

In [42]:
answer['answer']

'<pad>   pad>  1,200  +  from  40+  countries\n                                                                                                                                                                                                                                               '

In [48]:
print(answer['source_documents'])

[Document(page_content='a self-contained international community with a cosmopolitan approach to living and \nlearning. \n \nSince 1959, AIT has carried out its mission “to develop highly qualified and committed \nprofessionals who play a leading role in the region’s sustainable development and its \nintegration into the global economy” by supporting technological change and \nsustainable development through higher learning, research, capacity building and \noutreach. \n \nAIT’s renowned degree programs are administered by its School of Engineering and \nTechnology; School of Environment, Resources and Development; and School of \nManagement. Students benefit from challenging academic programs and exciting', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/NLP/A7/data/pdf/Student-Handbook_August-2021.pdf', 'page': 3, 'total_pages': 95, 'format': 'PDF 1.6', 'title': '', 'au

In [56]:
source = ''
for i in answer['source_documents']:
    source += i.page_content
print(source)

a self-contained international community with a cosmopolitan approach to living and 
learning. 
 
Since 1959, AIT has carried out its mission “to develop highly qualified and committed 
professionals who play a leading role in the region’s sustainable development and its 
integration into the global economy” by supporting technological change and 
sustainable development through higher learning, research, capacity building and 
outreach. 
 
AIT’s renowned degree programs are administered by its School of Engineering and 
Technology; School of Environment, Resources and Development; and School of 
Management. Students benefit from challenging academic programs and excitingAIT’s main campus offers fast and easy access to Bangkok, a city at the crossroads of 
East, Southeast and South Asia. AIT has also established a key learning center in 
Vietnam. 
 
With friends all over the globe, a strong history of academic excellence, and an 
enduring reputation for responding to emerging regional 